NAME- VARUN MISHRA

SCHOLAR NO- 24215011102

Q- Image Classification Using CIFAR-10 Dataset using simple deep network with 4 hidden layers and 3 dropout layer also apply pruning and quantization to reduce size and report size of model.
Write the code for google colab

In [ ]:
# Install the TensorFlow Model Optimization Toolkit
!pip install -q tensorflow-model-optimization

In [ ]:
# Import necessary libraries
import tensorflow as tf
import tensorflow_model_optimization as tfmot

print("TensorFlow version:", tf.__version__)
print("TensorFlow Model Optimization Toolkit version:", tfmot.__version__)

# It might be necessary to install a specific version if there's a compatibility issue
# !pip install tensorflow==2.x.x
# !pip install tensorflow-model-optimization==0.x.x


TensorFlow version: 2.18.0
TensorFlow Model Optimization Toolkit version: 0.8.0


In [ ]:
!pip uninstall tensorflow tensorflow-model-optimization -y
!pip install tensorflow tensorflow-model-optimization


Found existing installation: tensorflow 2.18.0
Uninstalling tensorflow-2.18.0:
  Successfully uninstalled tensorflow-2.18.0
Found existing installation: tensorflow-model-optimization 0.8.0
Uninstalling tensorflow-model-optimization-0.8.0:
  Successfully uninstalled tensorflow-model-optimization-0.8.0
  Using cached tensorflow_model_optimization-0.8.0-py2.py3-none-any.whl.metadata (904 bytes)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.4/615.4 MB 1.2 MB/s eta 0:00:00
Using cached tensorflow_model_optimization-0.8.0-py2.py3-none-any.whl (242 kB)


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.utils.prune as prune
import torch.quantization
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os
import copy

# Define transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load CIFAR-10 dataset
train_dataset = datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.CIFAR10(root='./data', train=False, transform=transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Define the neural network
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(32 * 32 * 3, 512)
        self.dropout1 = nn.Dropout(0.2)
        self.fc2 = nn.Linear(512, 256)
        self.dropout2 = nn.Dropout(0.2)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 64)
        self.dropout3 = nn.Dropout(0.2)
        self.fc5 = nn.Linear(64, 10)

    def forward(self, x):
        x = self.flatten(x)
        x = torch.relu(self.fc1(x))
        x = self.dropout1(x)
        x = torch.relu(self.fc2(x))
        x = self.dropout2(x)
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = self.dropout3(x)
        x = self.fc5(x)
        return x

# Initialize model and send it to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleNN().to(device)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train function
def train_model(model, epochs=10):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        avg_loss = total_loss / len(train_loader)
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}')

# Train the model
train_model(model, epochs=10)

# Function to calculate and print model size
def get_model_size(model, filename):
    torch.save(model.state_dict(), filename)
    size_kb = os.path.getsize(filename) / 1024
    print(f'{filename} size: {size_kb:.2f} KB')
    os.remove(filename)

# Original model size
get_model_size(model, 'original_model.pth')

# Create copies for independent pruning and quantization
pruned_model = copy.deepcopy(model)
quantized_model = copy.deepcopy(model)

# Apply pruning only to pruned_model
for name, module in pruned_model.named_modules():
    if isinstance(module, nn.Linear):
        prune.l1_unstructured(module, name='weight', amount=0.4)
        prune.remove(module, 'weight')

# Pruned model size
get_model_size(pruned_model, 'pruned_model.pth')

# Apply quantization only to quantized_model
quantized_model = torch.quantization.quantize_dynamic(
    quantized_model, {nn.Linear}, dtype=torch.qint8
)

# Quantized model size
get_model_size(quantized_model, 'quantized_model.pth')


Epoch [1/10], Loss: 1.7663
Epoch [2/10], Loss: 1.5845
Epoch [3/10], Loss: 1.4942
Epoch [4/10], Loss: 1.4329
Epoch [5/10], Loss: 1.3784
Epoch [6/10], Loss: 1.3344
Epoch [7/10], Loss: 1.2915
Epoch [8/10], Loss: 1.2571
Epoch [9/10], Loss: 1.2260
Epoch [10/10], Loss: 1.1971
original_model.pth size: 6826.03 KB
pruned_model.pth size: 6826.00 KB
quantized_model.pth size: 1715.34 KB
